In [1]:
## 3장에서 word2vec의 구조를 배우고 CBOW 모델을 구현했다.
## CBOW 모델은 단순한 2층 신경망이라서 간단하게 구현할 수 있었다. 그러나 말뭉치에 포함된 어휘 수가 많아지면
## 계산량이 어마어마하게 커진다는 점이다.
## 그래서 두 가지 개선을 추가한다.
## 첫 번째 개선은 Embedding 이라는 새로운 계층을 도입하는 것이다.
## 두 번째 개선은 Negative Sampling 이라는 새로운 손실함수를 도입하는 것이다.

# 1. word2vec 개선 - 1

In [3]:
## 어휘가 100만 개일 때를 가정한 CBOW 모델에서 두 계산이 병목이 된다.
## 1. 입력층의 원핫 표현과 가중치 행렬 W(in)의 곱 계산
## 어휘가 100만 개이므로 원핫 벡터의 크기가 100만이 된다. 또한 가중치와의 곱에서 엄청난 계산 자원을 사용한다.
## 2. 은닉층과 가중치 행렬 W(out)의 곱 및 Softmax 계층의 계산

In [4]:
## 1번 병목은 Embedding 계층을 도입하는 것으로 해결하고
## 2번 병목은 Negative Sampling을 도입하는 것으로 해결한다.

### Embedding 계층

In [5]:
## 사실 원핫 벡터와 가중치의 곱은 단지 원핫에서 1에 해당하는 벡터와의 곱을 수행하는 특정 행을 추출하는 것이다.
## 따라서 원핫 표현으로의 변환과 MatMul 계층의 행렬 곱 계산은 사실 필요없다.
## 그래서 가중치 매개변수로부터'단어 ID에 해당하는 행(벡터)'을 추출하는 계층을 만든다.
## 이 계층을 Embedding 이라고 한다.
## Embedding 계층에 '단어 임베딩(분산 표현)'을 저장하는 것이다.

## 자연어 처리 분야에서 단어의 밀집벡터 표현을 '단어 임베딩' 혹은 '분산 표현'이라 한다.

### Embedding 계층 구현

In [6]:
import numpy as np
W = np.arange(21).reshape(7, 3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [11]:
W[2] # 가중치로부터 특정 행 추출 가능

array([6, 7, 8])

In [12]:
W[4]

array([12, 13, 14])

In [14]:
idx = np.array([1, 0, 3, 0])
W[idx] ## 인수에 배열을 사용하면 가중치로부터 여러 행 한 번에 추출 가능

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [16]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0 ## 좋지 않은 예
        dW[self.idx] = dout
        return None

In [18]:
## 앞의 backward() 구현에는 사실 문제가 존재한다.
## idx의 원소가 중복될 때 발생한다. 그렇게 되면 먼저 쓰여진 값을 덮어쓰게 된다.
## 이 중복문제를 해결하려면 할당이 아닌 더하기를 해야한다.

In [19]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
            
        return None

In [21]:
## 일반적으로 파이썬에서 for문보다는 넘파이의 내장 메서드를 사용하는 편이 더 빠르다.
## 넘파이의 메서드에는 속도와 효율을 높여주는 최적화가 적용돼 있기 때문이다. 

# 2. word2vec 개선 - 2

### 은닉층 이후 계산의 문제점

In [24]:
## 앞에서 Embedding 계층을 도입하여 입력층 계산에서의 낭비를 줄였다.
## 남은 문제는 은닉층 이후의 처리이다. (은닉층 뉴런과 가중치의 곱, softmax 계층 계산)

## 은닉층 뉴런과 가중치의 곱 - 거대한 행렬을 곱하는 문제
## softmax 계층 계산 - 분모를 얻으려면 어휘 수만큼 exp() 연산을 해야한다

### 다중 분류에서 이진 분류로

In [25]:
## 네거티브 샘플링 기법의 핵심 아이디어는 '이진 분류'에 있다. 
## 정확하게는 '다중 분류'를 '이진 분류'로 근사하는 것이 네거티브 샘플링의 포인트다.

## 지금까지는 맥락이 주어지면 정답이 되는 단어를 높은 확률로 추측하도록 만드는 일을 했다.
## 이제부터 생각해야 할 것은 'you'와 'goodbye'가 맥락으로 주어질 때, 타깃이 'say'입니까? 라는 질문에 답하는 신경망이다.
## 이렇게 하면 출력층에는 뉴런을 하나만 준비하면 된다.(타깃 단어만의 점수를 구하는 신경망)

## 따라서 은닉층과 출력 측의 가중치 행렬의 내적은 'say'에 해당하는 열만 가중치에서 추출하여 은닉층과 내적하는 것과 같다.

### 시그모이드 함수와 교차 엔트로피 오차

In [26]:
## 이진 분류 문제를 신경망으로 풀려면 점수에 시그모이드 함수를 적용해 확률로 변환하고, 손실을 구할 때는 교차 엔트로피를 사용한다.

## 다중분류 : softmax + cross-entropy-error
## 이진분류 : sigmoid + cross-entropy-error

### Embedding Dot 계층

In [27]:
## Embedding Dot 계층은 Embedding 계층과 Dot(내적)의 처리를 합친 계층이다.
## 이 계층을 사용하면 은닉층 이후의 처리가 간단해진다.

In [44]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis = 1)
        
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

### 네거티브 샘플링

In [45]:
## 지금까지 다중분류를 이진분류로 변환할 수 있었다. 하지만 이것만으로는 문제가 해결되지 않는다.
## 지금까지는 정답에 대해서만 학습을 했기 때문에 정답 이외의 단어에 대해서는 어떠한 지식도 획득하지 못한다.

## 우리가 바라는 것은 정답('say')에 대해서는 sigmoid 계층의 출력을 1에 가깝게 만들고, 오답('say')이외의 단어에 대해서는
## sigmoid 계층의 출력을 0에 가깝게 만드는 것이다. 

In [46]:
## 그러면 모든 단어를 대상으로 하여 이진 분류를 학습 시키는 건 어떨까?
## 이는 어휘의 수가 많아지면 감당하 수 없게 되므로 좋지 못하다.

## 그래서 근사적 해법으로 정답이 아닌 단어를 몇 개 선택한다.
## 즉, 적은 수의 오답 예를 샘플링해 사용한다. 이것이 바로 네거티브 샘플링 기법이 의미하는 것이다.

## 정리하면, 네거티브 샘플링 기법은 정답을 타깃으로 한 경우의 손실을 구하고, 그와 동시에 오답 예를 몇 개 샘플링 해서 
## 그것에 대해서도 손실을 구한다. 그리고 각각의 데이터의 손실을 더한 값을 최종 손실로 한다.

### 네거티브 샘플링의 샘플링 기법

In [49]:
## 그렇다면 오답인 단어를 어떻게 샘플링 하느냐 하는 문제가 남는다.
## 단순히 무작위로 샘플링 하는 것보다 좋은 방법이 있다. 바로 말뭉치의 통계 데이터를 기초로 샘플링 하는 방법이다. 
## 한마디로 말뭉치에서 자주 등장하는 단어를 많이 추출하고 드물게 등장하는 단어를 적게 추출한다.
## 단어의 빈도를 기준으로 샘플링하려면 먼저 말뭉치에서 각 단어의 출현 횟수를 구해 '확률분포'로 나타낸다.
## 그런다음 그 확률 분포대로 단어를 샘플링하면 된다.

In [2]:
## 확률분포대로 샘플링하면 자주 등장하는 단어는 선택될 확률이 높고, 희소한 단어는 선택되기 어렵다
## 네거티브 샘플링에서 부정적 예를 가능한 많이 다루는 것이 좋지만, 계산량 문제 때문에 적은 수로 한정해야 한다.
## 그런데 우연히 희소한 단어만 선택된다면 어떻게 될까?
## 당연히 결과가 나빠진다. 즉, 드문 단어를 잘 처리하는 일은 중요도가 낮다. 그보다는 흔한 단어를 잘 처리하는 편이 좋은 결과로
## 이어진다.

In [4]:
## 확률분포에 따라 샘플링하는 코드를 구현해보자

import numpy as np

## 0에서 9까지의 숫자 중 하나를 무작위로 샘플링
np.random.choice(10)


2

In [5]:
# words에서 하나만 무작위로 샘플링
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
np.random.choice(words)

'hello'

In [7]:
# 5개만 무작위로 샘플링( 중복 0 )
np.random.choice(words, size = 5)

array(['you', 'hello', 'say', 'goodbye', 'goodbye'], dtype='<U7')

In [8]:
# 5개만 무작위로 샘플링( 중복 x )
np.random.choice(words, size = 5, replace = False)

array(['.', 'say', 'hello', 'you', 'goodbye'], dtype='<U7')

In [10]:
# 확률분포에 따라 샘플링
P = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]

np.random.choice(words, p = P)

'say'

In [11]:
## 이제 이 함수를 바탕으로 부정적 예를 샘플링하기만 하면 된다.
## 그런데 word2vec의 네거티브 샘플링에서는 앞의 확률 분포에 0.75제곱을 하라고 권고한다.
## 이유는 출현 확률이 낮은 단어를 버리지 않기 위해서이다.
## 0.75 제곱을 함으로써, 원래 확률이 낮은 단어의 확률을 살짝 높일 수 있다. 

In [13]:
P = [0.7, 0.29, 0.01]
new_p = np.power(P, 0.75)
new_p /= np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


In [20]:
## 말뭉치에서 단어의 확률분포를 만들고, 0.75를 제곱한 다음, np.random.choice()를 사용해 부정적 예를 샘플링 한다.
## 이 처리를 담당하는 클래스를 UnigramSampler라는 이름으로 구현

# coding: utf-8
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections

class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size), replace=True, p=self.word_p)

        return negative_sample

In [21]:
## UnigramSampler 클래스 사용

corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[4 2]
 [1 0]
 [2 1]]


In [22]:
## get_negative_sample(target) 메서드는 target 인수로 지정한 단어를 정답으로 해석하고, 그 외 단어 ID를 샘플링한다.
## 여기서는 긍정적인 예로[1, 3, 0]이라는 3개의 데이터를 미니배치로 다뤘다.
## 이 각각의 데이터에 대해 부정적 예를 2개씩 샘플링 한다.
## 이제 우리는 부정적 예를 샘플링할 수 있게 되었다.

### 네거티브 샘플링 구현

In [25]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power = 0.75, sample_size = 5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layer = [EmbeddingDot(W) for _ in range(sample_size + 1)]
        self.params, self.grads = [], []
        
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def forwarad(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)
        # forward(h, target)의 인수는 은닉층 뉴런 h와 긍정적 예의 타깃 target이다.
        
        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype = np.int32) # 긍정적 예의 정답 label은 1이다.
        loss = self.loss_layers[0].forward(score, correct_label)
        
        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype = np.int32) # 부정적 예의 정답 label은 0이다.
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)
            
        return loss
    
    def backward(self, dout = 1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)
            
        return dh

In [26]:
## 이거 코드만 쳐본다고 될 게 아니라 전부 그래프 하나하나 그려가면서 비교하면서 코드 짜봐야 의미가 있을 거 같은데

# 3. 개선판 word2vec 학습

In [27]:
## 지금까지 기존 word2vec을 개선했다. 
## 그럼 이러한 개선을 신경망 구현에 적용해보자

### CBOW 모델 구현

In [1]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')
        
        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power = 0.75, sample_size = 5)
        
        # 모든 가중치와 기울기를 배열에 모은다
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        # 인스턴스 변수에 단어의 분산 표현을 저장한다
        self.word_vecs = W_in
        
    def forward(self, contexts, target):
        h = 0 
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss
    
    def backward(self, dout = 1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

### CBOW 모델 학습 코드

In [4]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from ch04.cbow import CBOW
from ch04.skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)


Done
| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 2[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 4[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 6[s] | 손실 4.12
| 에폭 1 |  반복 81 / 9295 | 시간 9[s] | 손실 4.04
| 에폭 1 |  반복 101 / 9295 | 시간 11[s] | 손실 3.92
| 에폭 1 |  반복 121 / 9295 | 시간 13[s] | 손실 3.78
| 에폭 1 |  반복 141 / 9295 | 시간 15[s] | 손실 3.62
| 에폭 1 |  반복 161 / 9295 | 시간 18[s] | 손실 3.48
| 에폭 1 |  반복 181 / 9295 | 시간 20[s] | 손실 3.35
| 에폭 1 |  반복 201 / 9295 | 시간 22[s] | 손실 3.25
| 에폭 1 |  반복 221 / 9295 | 시간 24[s] | 손실 3.15
| 에폭 1 |  반복 241 / 9295 | 시간 27[s] | 손실 3.09
| 에폭 1 |  반복 261 / 9295 | 시간 29[s] | 손실 3.03
| 에폭 1 |  반복 281 / 9295 | 시간 31[s] | 손실 2.95
| 에폭 1 |  반복 301 / 9295 | 시간 33[s] | 손실 2.90
| 에폭 1 |  반복 321 / 9295 | 시간 35[s] | 손실 2.87
| 에폭 1 |  반복 341 / 9295 | 시간 38[s] | 손실 2.84
| 에폭 1 |  반복 361 / 9295 | 시간 40[s] | 손실 2.81
| 에폭 1 |  반복 381 / 9295 | 시간 42[s] | 손실 2.81
| 에폭 1 |  반복 401 / 9295 | 시간 44[s] | 손실 2.77
| 에폭 1 |  반복 421 / 9295 | 시간 46[s] | 손실 2.74
| 에폭 1 |  반복 441

| 에폭 1 |  반복 3521 / 9295 | 시간 392[s] | 손실 2.44
| 에폭 1 |  반복 3541 / 9295 | 시간 394[s] | 손실 2.40
| 에폭 1 |  반복 3561 / 9295 | 시간 396[s] | 손실 2.39
| 에폭 1 |  반복 3581 / 9295 | 시간 399[s] | 손실 2.43
| 에폭 1 |  반복 3601 / 9295 | 시간 401[s] | 손실 2.43
| 에폭 1 |  반복 3621 / 9295 | 시간 403[s] | 손실 2.42
| 에폭 1 |  반복 3641 / 9295 | 시간 405[s] | 손실 2.43
| 에폭 1 |  반복 3661 / 9295 | 시간 407[s] | 손실 2.43
| 에폭 1 |  반복 3681 / 9295 | 시간 410[s] | 손실 2.44
| 에폭 1 |  반복 3701 / 9295 | 시간 412[s] | 손실 2.42
| 에폭 1 |  반복 3721 / 9295 | 시간 414[s] | 손실 2.42
| 에폭 1 |  반복 3741 / 9295 | 시간 416[s] | 손실 2.42
| 에폭 1 |  반복 3761 / 9295 | 시간 418[s] | 손실 2.42
| 에폭 1 |  반복 3781 / 9295 | 시간 421[s] | 손실 2.40
| 에폭 1 |  반복 3801 / 9295 | 시간 423[s] | 손실 2.39
| 에폭 1 |  반복 3821 / 9295 | 시간 425[s] | 손실 2.41
| 에폭 1 |  반복 3841 / 9295 | 시간 427[s] | 손실 2.38
| 에폭 1 |  반복 3861 / 9295 | 시간 430[s] | 손실 2.40
| 에폭 1 |  반복 3881 / 9295 | 시간 432[s] | 손실 2.43
| 에폭 1 |  반복 3901 / 9295 | 시간 434[s] | 손실 2.41
| 에폭 1 |  반복 3921 / 9295 | 시간 436[s] | 손실 2.41
| 에폭 1 |  반복 

| 에폭 1 |  반복 7021 / 9295 | 시간 779[s] | 손실 2.30
| 에폭 1 |  반복 7041 / 9295 | 시간 781[s] | 손실 2.26
| 에폭 1 |  반복 7061 / 9295 | 시간 783[s] | 손실 2.26
| 에폭 1 |  반복 7081 / 9295 | 시간 785[s] | 손실 2.29
| 에폭 1 |  반복 7101 / 9295 | 시간 788[s] | 손실 2.27
| 에폭 1 |  반복 7121 / 9295 | 시간 790[s] | 손실 2.26
| 에폭 1 |  반복 7141 / 9295 | 시간 792[s] | 손실 2.31
| 에폭 1 |  반복 7161 / 9295 | 시간 794[s] | 손실 2.29
| 에폭 1 |  반복 7181 / 9295 | 시간 797[s] | 손실 2.27
| 에폭 1 |  반복 7201 / 9295 | 시간 799[s] | 손실 2.26
| 에폭 1 |  반복 7221 / 9295 | 시간 801[s] | 손실 2.27
| 에폭 1 |  반복 7241 / 9295 | 시간 804[s] | 손실 2.24
| 에폭 1 |  반복 7261 / 9295 | 시간 806[s] | 손실 2.29
| 에폭 1 |  반복 7281 / 9295 | 시간 808[s] | 손실 2.28
| 에폭 1 |  반복 7301 / 9295 | 시간 810[s] | 손실 2.30
| 에폭 1 |  반복 7321 / 9295 | 시간 812[s] | 손실 2.29
| 에폭 1 |  반복 7341 / 9295 | 시간 815[s] | 손실 2.30
| 에폭 1 |  반복 7361 / 9295 | 시간 817[s] | 손실 2.28
| 에폭 1 |  반복 7381 / 9295 | 시간 819[s] | 손실 2.28
| 에폭 1 |  반복 7401 / 9295 | 시간 821[s] | 손실 2.25
| 에폭 1 |  반복 7421 / 9295 | 시간 823[s] | 손실 2.26
| 에폭 1 |  반복 

| 에폭 2 |  반복 1201 / 9295 | 시간 1166[s] | 손실 2.12
| 에폭 2 |  반복 1221 / 9295 | 시간 1168[s] | 손실 2.12
| 에폭 2 |  반복 1241 / 9295 | 시간 1170[s] | 손실 2.18
| 에폭 2 |  반복 1261 / 9295 | 시간 1172[s] | 손실 2.13
| 에폭 2 |  반복 1281 / 9295 | 시간 1174[s] | 손실 2.13
| 에폭 2 |  반복 1301 / 9295 | 시간 1177[s] | 손실 2.17
| 에폭 2 |  반복 1321 / 9295 | 시간 1179[s] | 손실 2.19
| 에폭 2 |  반복 1341 / 9295 | 시간 1181[s] | 손실 2.13
| 에폭 2 |  반복 1361 / 9295 | 시간 1183[s] | 손실 2.18
| 에폭 2 |  반복 1381 / 9295 | 시간 1186[s] | 손실 2.16
| 에폭 2 |  반복 1401 / 9295 | 시간 1188[s] | 손실 2.14
| 에폭 2 |  반복 1421 / 9295 | 시간 1190[s] | 손실 2.16
| 에폭 2 |  반복 1441 / 9295 | 시간 1192[s] | 손실 2.13
| 에폭 2 |  반복 1461 / 9295 | 시간 1195[s] | 손실 2.15
| 에폭 2 |  반복 1481 / 9295 | 시간 1197[s] | 손실 2.16
| 에폭 2 |  반복 1501 / 9295 | 시간 1199[s] | 손실 2.17
| 에폭 2 |  반복 1521 / 9295 | 시간 1201[s] | 손실 2.15
| 에폭 2 |  반복 1541 / 9295 | 시간 1203[s] | 손실 2.20
| 에폭 2 |  반복 1561 / 9295 | 시간 1206[s] | 손실 2.12
| 에폭 2 |  반복 1581 / 9295 | 시간 1208[s] | 손실 2.14
| 에폭 2 |  반복 1601 / 9295 | 시간 1210[s] | 

| 에폭 2 |  반복 4621 / 9295 | 시간 1544[s] | 손실 2.08
| 에폭 2 |  반복 4641 / 9295 | 시간 1547[s] | 손실 2.10
| 에폭 2 |  반복 4661 / 9295 | 시간 1549[s] | 손실 2.06
| 에폭 2 |  반복 4681 / 9295 | 시간 1551[s] | 손실 2.06
| 에폭 2 |  반복 4701 / 9295 | 시간 1553[s] | 손실 2.10
| 에폭 2 |  반복 4721 / 9295 | 시간 1555[s] | 손실 2.09
| 에폭 2 |  반복 4741 / 9295 | 시간 1558[s] | 손실 2.09
| 에폭 2 |  반복 4761 / 9295 | 시간 1560[s] | 손실 2.10
| 에폭 2 |  반복 4781 / 9295 | 시간 1562[s] | 손실 2.07
| 에폭 2 |  반복 4801 / 9295 | 시간 1564[s] | 손실 2.10
| 에폭 2 |  반복 4821 / 9295 | 시간 1566[s] | 손실 2.10
| 에폭 2 |  반복 4841 / 9295 | 시간 1569[s] | 손실 2.11
| 에폭 2 |  반복 4861 / 9295 | 시간 1571[s] | 손실 2.08
| 에폭 2 |  반복 4881 / 9295 | 시간 1573[s] | 손실 2.11
| 에폭 2 |  반복 4901 / 9295 | 시간 1575[s] | 손실 2.10
| 에폭 2 |  반복 4921 / 9295 | 시간 1578[s] | 손실 2.11
| 에폭 2 |  반복 4941 / 9295 | 시간 1580[s] | 손실 2.06
| 에폭 2 |  반복 4961 / 9295 | 시간 1582[s] | 손실 2.07
| 에폭 2 |  반복 4981 / 9295 | 시간 1584[s] | 손실 2.07
| 에폭 2 |  반복 5001 / 9295 | 시간 1586[s] | 손실 2.11
| 에폭 2 |  반복 5021 / 9295 | 시간 1589[s] | 

| 에폭 2 |  반복 8041 / 9295 | 시간 1923[s] | 손실 2.03
| 에폭 2 |  반복 8061 / 9295 | 시간 1925[s] | 손실 2.03
| 에폭 2 |  반복 8081 / 9295 | 시간 1927[s] | 손실 2.01
| 에폭 2 |  반복 8101 / 9295 | 시간 1930[s] | 손실 2.06
| 에폭 2 |  반복 8121 / 9295 | 시간 1932[s] | 손실 1.99
| 에폭 2 |  반복 8141 / 9295 | 시간 1934[s] | 손실 2.04
| 에폭 2 |  반복 8161 / 9295 | 시간 1936[s] | 손실 2.03
| 에폭 2 |  반복 8181 / 9295 | 시간 1938[s] | 손실 2.03
| 에폭 2 |  반복 8201 / 9295 | 시간 1941[s] | 손실 2.00
| 에폭 2 |  반복 8221 / 9295 | 시간 1943[s] | 손실 2.04
| 에폭 2 |  반복 8241 / 9295 | 시간 1945[s] | 손실 2.03
| 에폭 2 |  반복 8261 / 9295 | 시간 1947[s] | 손실 2.01
| 에폭 2 |  반복 8281 / 9295 | 시간 1949[s] | 손실 2.04
| 에폭 2 |  반복 8301 / 9295 | 시간 1952[s] | 손실 2.03
| 에폭 2 |  반복 8321 / 9295 | 시간 1954[s] | 손실 2.03
| 에폭 2 |  반복 8341 / 9295 | 시간 1956[s] | 손실 2.04
| 에폭 2 |  반복 8361 / 9295 | 시간 1958[s] | 손실 2.00
| 에폭 2 |  반복 8381 / 9295 | 시간 1961[s] | 손실 2.03
| 에폭 2 |  반복 8401 / 9295 | 시간 1963[s] | 손실 2.03
| 에폭 2 |  반복 8421 / 9295 | 시간 1965[s] | 손실 2.03
| 에폭 2 |  반복 8441 / 9295 | 시간 1967[s] | 

| 에폭 3 |  반복 2181 / 9295 | 시간 2303[s] | 손실 1.94
| 에폭 3 |  반복 2201 / 9295 | 시간 2305[s] | 손실 1.94
| 에폭 3 |  반복 2221 / 9295 | 시간 2307[s] | 손실 1.91
| 에폭 3 |  반복 2241 / 9295 | 시간 2310[s] | 손실 1.93
| 에폭 3 |  반복 2261 / 9295 | 시간 2312[s] | 손실 1.95
| 에폭 3 |  반복 2281 / 9295 | 시간 2314[s] | 손실 1.93
| 에폭 3 |  반복 2301 / 9295 | 시간 2316[s] | 손실 1.97
| 에폭 3 |  반복 2321 / 9295 | 시간 2319[s] | 손실 1.93
| 에폭 3 |  반복 2341 / 9295 | 시간 2321[s] | 손실 1.94
| 에폭 3 |  반복 2361 / 9295 | 시간 2323[s] | 손실 1.96
| 에폭 3 |  반복 2381 / 9295 | 시간 2325[s] | 손실 1.95
| 에폭 3 |  반복 2401 / 9295 | 시간 2327[s] | 손실 1.94
| 에폭 3 |  반복 2421 / 9295 | 시간 2330[s] | 손실 1.93
| 에폭 3 |  반복 2441 / 9295 | 시간 2332[s] | 손실 1.93
| 에폭 3 |  반복 2461 / 9295 | 시간 2334[s] | 손실 1.93
| 에폭 3 |  반복 2481 / 9295 | 시간 2336[s] | 손실 1.94
| 에폭 3 |  반복 2501 / 9295 | 시간 2338[s] | 손실 1.93
| 에폭 3 |  반복 2521 / 9295 | 시간 2341[s] | 손실 1.95
| 에폭 3 |  반복 2541 / 9295 | 시간 2343[s] | 손실 1.93
| 에폭 3 |  반복 2561 / 9295 | 시간 2345[s] | 손실 1.92
| 에폭 3 |  반복 2581 / 9295 | 시간 2347[s] | 

| 에폭 3 |  반복 5601 / 9295 | 시간 2683[s] | 손실 1.88
| 에폭 3 |  반복 5621 / 9295 | 시간 2685[s] | 손실 1.91
| 에폭 3 |  반복 5641 / 9295 | 시간 2687[s] | 손실 1.90
| 에폭 3 |  반복 5661 / 9295 | 시간 2689[s] | 손실 1.91
| 에폭 3 |  반복 5681 / 9295 | 시간 2691[s] | 손실 1.88
| 에폭 3 |  반복 5701 / 9295 | 시간 2694[s] | 손실 1.90
| 에폭 3 |  반복 5721 / 9295 | 시간 2696[s] | 손실 1.94
| 에폭 3 |  반복 5741 / 9295 | 시간 2698[s] | 손실 1.92
| 에폭 3 |  반복 5761 / 9295 | 시간 2700[s] | 손실 1.91
| 에폭 3 |  반복 5781 / 9295 | 시간 2702[s] | 손실 1.95
| 에폭 3 |  반복 5801 / 9295 | 시간 2705[s] | 손실 1.92
| 에폭 3 |  반복 5821 / 9295 | 시간 2707[s] | 손실 1.95
| 에폭 3 |  반복 5841 / 9295 | 시간 2709[s] | 손실 1.90
| 에폭 3 |  반복 5861 / 9295 | 시간 2711[s] | 손실 1.88
| 에폭 3 |  반복 5881 / 9295 | 시간 2713[s] | 손실 1.92
| 에폭 3 |  반복 5901 / 9295 | 시간 2716[s] | 손실 1.89
| 에폭 3 |  반복 5921 / 9295 | 시간 2718[s] | 손실 1.91
| 에폭 3 |  반복 5941 / 9295 | 시간 2720[s] | 손실 1.93
| 에폭 3 |  반복 5961 / 9295 | 시간 2722[s] | 손실 1.89
| 에폭 3 |  반복 5981 / 9295 | 시간 2724[s] | 손실 1.91
| 에폭 3 |  반복 6001 / 9295 | 시간 2727[s] | 

| 에폭 3 |  반복 9021 / 9295 | 시간 3059[s] | 손실 1.92
| 에폭 3 |  반복 9041 / 9295 | 시간 3062[s] | 손실 1.90
| 에폭 3 |  반복 9061 / 9295 | 시간 3064[s] | 손실 1.91
| 에폭 3 |  반복 9081 / 9295 | 시간 3066[s] | 손실 1.91
| 에폭 3 |  반복 9101 / 9295 | 시간 3068[s] | 손실 1.90
| 에폭 3 |  반복 9121 / 9295 | 시간 3070[s] | 손실 1.86
| 에폭 3 |  반복 9141 / 9295 | 시간 3073[s] | 손실 1.87
| 에폭 3 |  반복 9161 / 9295 | 시간 3075[s] | 손실 1.90
| 에폭 3 |  반복 9181 / 9295 | 시간 3077[s] | 손실 1.91
| 에폭 3 |  반복 9201 / 9295 | 시간 3079[s] | 손실 1.87
| 에폭 3 |  반복 9221 / 9295 | 시간 3081[s] | 손실 1.90
| 에폭 3 |  반복 9241 / 9295 | 시간 3084[s] | 손실 1.89
| 에폭 3 |  반복 9261 / 9295 | 시간 3086[s] | 손실 1.85
| 에폭 3 |  반복 9281 / 9295 | 시간 3088[s] | 손실 1.92
| 에폭 4 |  반복 1 / 9295 | 시간 3090[s] | 손실 1.91
| 에폭 4 |  반복 21 / 9295 | 시간 3092[s] | 손실 1.78
| 에폭 4 |  반복 41 / 9295 | 시간 3094[s] | 손실 1.81
| 에폭 4 |  반복 61 / 9295 | 시간 3096[s] | 손실 1.84
| 에폭 4 |  반복 81 / 9295 | 시간 3099[s] | 손실 1.83
| 에폭 4 |  반복 101 / 9295 | 시간 3101[s] | 손실 1.84
| 에폭 4 |  반복 121 / 9295 | 시간 3103[s] | 손실 1.80
| 에폭 

KeyboardInterrupt: 

### CBOW 모델 평가

In [5]:
# coding: utf-8
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle


pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# 가장 비슷한(most similar) 단어 뽑기
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

# 유추(analogy) 작업
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)


FileNotFoundError: [Errno 2] No such file or directory: 'cbow_params.pkl'

# word2vec 남은 주제

### word2vec을 사용한 애플리케이션의 예

In [7]:
## word2vec으로 얻은 단어의 분산표현은 비슷한 단어를 찾는 용도로 이용할 수 있다.
## 그러나 단어의 분산표현의 장점은 여기서 끝이 아니다
## 자연어 처리 분야에서 단어의 분산표현이 중요한 이유는 전이학습(transfer learning)에 있다.
## 자연어 문제를 풀 때 word2vec의 단어 분산 표현을 처음부터 학습하는 일은 거의 없다.
## 그 대신 먼저 큰 말뭉치(위키백과나 구글 뉴스의 텍스트 데이터 등)로 학습을 끝낸 후, 그 분산 표현을 각자의 작업에 이용하는 것이다.
## 예컨데 텍스트 분류, 문서 클러스터링, 품사 태크 달기, 감정분석 등 자연어 처리 작업이라면 가장 먼저 단어를 벡터로 변환하는 작업을
## 해야 하는데, 이때 학습을 미리 끝낸 단어의 분산 표현을 이용할 수 있다. 

In [8]:
## 단어의 분산표현은 단어를 고정 길이 벡터로 변환해준다는 장점도 있다.
## 게다가 문장도 단어의 분산 표현을 사용하여 고정길이 벡터로 변환할 수 있다.
## 문장을 고정 길이 벡터로 변환하는 방법은 활발하게 연구되고 있는데, 가장 간단한 방법은 문장의 각 단어를 분산 표현으로 변환하고
## 그 합을 구하는 것이다. 이를 bag-of-words라 하여, 단어의 순서를 고려하지 않는 모델이다. 
## 또 5장에서 설명하는 순환 신경망을 사용하면 한층 세련된 방법으로 문장을 고정 길이 벡터로 변환할 수 있다.

In [9]:
## 단어나 문장을 고정 길이 벡터로 변환할 수 있다는 점은 매우 중요하다.
## 자연어를 벡터로 변환할 수 있다면 일반적인 머신러닝 기법을 적용할 수 있기 때문이다.

In [10]:
## 질문(자연어) --> 단어 벡터화(word2vec) --> 머신러닝 시스템(신경망, SVM 등) --> 답변

In [11]:
## 위의 파이프라인에서는 단어의 분산표현학습과 머신러닝 시스템의 학습은 서로 다른 데이터셋을 사용해 개별적으로 수행하는 것이
## 일반적이다. 예컨대 위키백과 같은 범용 말뭉치를 사용해 미리 단어 분산 표현을 학습하고 현재 직면한 문제에 관련하여 수집한
## 데이터를 가지고 머신러닝 시스템을 학습시킨다.
## 그러나 직면한 문제의 학습 데이터가 아주 많다면, 단어의 분산표현과 머신러닝 시스템 학습 모두를 처음부터 수행하는 방안도
## 고려해볼 수 있다. 

In [12]:
## 정리하면 자연어를 다루는 문제는 단어의 분산표현이라는 방법으로 벡터화할 수 있다.
## 그 덕에 일반적인 머신러닝 기법으로 해결할 수 있게 된다.
## 게다가 word2vec로 전이학습의 혜택을 누릴 수 있다.

### 단어 벡터 평가 방법

In [13]:
## word2vec을 통해 단어의 분산 표현을 얻을 수 있었다. 그러면 그 분산 표현이 좋은지는 어떻게 평가할까?
## 단어의 분산 표현의 우수성은 실제 애플리케이션과는 분리해 평가하는 것이 일반적이다.
## 이때 자주 사용되는 평가 척도가 단어의 '유사성'이나 '유추 문제'를 활용한 평가이다.

In [14]:
## 유사성 평가에서는 사람이 작성한 단어 유사도를 검증 세트를 사용해 평가하는 것이 일반적이다.
## 사람이 단어 사이의 유사한 정도를 점수로 규정하고 그 점수와 word2vec에 의한 코사인 유사도 점수를 비교해 상관성을 보는 것이다.

## 유추 문제를 활용한 평가는 king : queen = man : ? 와 같은 유추 문제를 출제하고, 그 정답률로 우수성을 측정한다.
## 유추 문제를 이용하면 단어의 의미나 문법적인 문제를 제대로 이해하고 있는지를 측정할 수 있다.
## 그러므로 유추 문제를 풀 수 있는 단어의 분산 표현이라면 자연어를 다루는 애플리케이션에서도 좋은 결과를 기대할 수 있다.
## 그러나 애플리케이션 종류나 말충치의 내용 등, 다루는 문제 상황에 따라 달라지므로 유추 문제에 의한 평가가 높다고 해서
## 여러분의 애플리케이션에서도 반드시 좋은 결과가 나오리라는 보장은 없다.